In [5]:
from base64 import b64decode, b64encode
from Crypto import Random
import imageio
import cv2
import shutil
import time
import sys, os

import aes
import dwt
from Huff import HuffmanCoding

In [14]:
import pkg_resources

In [18]:
.__version__

DistributionNotFound: The 'shutil' distribution was not found and is required by the application

In [2]:
names = ["a", "h", "v", "d"]

In [3]:
def encryptMain(image, key, mode):
    nonce=["","","",""]
    data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    coeffs_from_arr = dwt.ProcesoDWT(data)
    dwt.Save_txt(coeffs_from_arr)
    h = HuffmanCoding()
    for cr in range(len(names)):
        path = "{}.txt".format(names[cr])
        h.compress(path)
        n=names[cr]
        with open(f"{n}_c.bin","rb") as file, open(f"{n}_cl.bin","wb") as out:
            plaintext = b64encode(file.read())
            appended = aes.saturar(plaintext)
            #ciphertext, nonce[cr]= aes.encrypt(appended,key,mode)
            ciphertext = aes.encrypt(appended,key,mode)
            out.write(ciphertext)
        os.remove(f"{n}.txt")
        #os.remove(f"{n}_c.bin")
    #return nonce

In [4]:
def decryptMain(imageName, key, mode, nonce=None):
    h = HuffmanCoding()
    for cr in range(len(names)):
        n =names[cr]
        with open("{}_cl.bin".format(n),"rb") as file, open("{}_c.bin".format(n),"wb") as out:
            if mode == "ECB":
                decrypted = aes.decrypt(file.read(),key,mode)
            else:
                decrypted = aes.decrypt(file.read(),key,mode, nonce[cr])
            satured= aes.remove_space_padding(decrypted)
            plaintextout = b64decode(satured)
            out.write(plaintextout)
        path = "{}_c.bin".format(names[cr])
        h.decompress(path)
    name_recon = dwt.ReconstructDWT()
    imageio.imwrite('{}_r.png'.format(imageName), name_recon)
    for trash in names:
            os.remove(f"{trash}.txt")
            os.remove(f"{trash}d.txt")
            os.remove(f"{trash}_c.bin")

In [17]:
dir="Images/"
name ="barbara.png"
key = Random.new().read(16)

In [18]:
encryptMain(dir+name,key,"ECB")

47.446 %
7.218 s
48.161 %
8.016 s
48.165 %
6.989 s
48.157 %
7.455 s


In [ ]:
decryptMain("lena",key , "ECB")

In [5]:
dir="Images/"
for files in os.listdir(dir):
    file = dir+files
    if file[-4:] != ".png":
        continue
    key = Random.new().read(16)
    start=time.time()
    print("I started "+files)
    #nonce = encryptMain(file,key, mode="OCB")
    encryptMain(file,key, mode="ECB")
    midle=time.time()
    try:
        #decryptMain(files[:-4],key,mode="OCB", nonce=nonce)
        decryptMain(files[:-4],key,mode="ECB")
        end= time.time()
    except:
        enc = midle - start
        print(file+" has failed")
        enc = midle - start
        f=open(f"{files[:-4]}.txt","w")
        f.write("Title: "+file+"\n")
        f.write(f"key: {key}")
        #f.write(f"nonce: {nonce}")
        f.write(f"Time encrypted: {enc}\n")
        f.close()
        continue
        
    enc = midle - start
    des = end - midle
    
    f=open(f"{files[:-4]}.txt","w")
    f.write("Title: "+file+"\n")
    f.write(f"key: {key}")
    #f.write(f"nonce: {nonce}")
    f.write(f"Time encrypted: {enc}\n")
    f.write(f"Time encrypted: {des}\n")
    total=end - start
    f.write(f"Total: {total}")
    f.close()
    if not os.path.exists(file[:-4]):
        os.makedirs(file[:-4])
    for clf in names:
        shutil.move(f"{clf}_cl.bin",file[:-4])
    shutil.move(f"{files[:-4]}_r.png",file[:-4])
    shutil.move(file,file[:-4])
    shutil.move(f"{file[:-1]}.txt",file[:-4])

I started barbara.png
47.446 %
7.314 s
48.161 %
7.933 s
48.165 %
7.55 s
48.157 %
7.491 s
11.289 s
11.603 s
12.602 s
12.382 s


I started black.png
17.023 %
0.229 s
17.023 %
0.308 s
17.023 %
0.315 s
17.023 %
0.312 s
0.662 s
0.581 s
0.61 s
0.876 s
Images/black.png has failed
I started color.png
29.397 %
12.909 s
44.416 %
25.308 s
45.655 %
25.609 s
45.654 %
26.252 s
19.116 s
34.907 s
40.634 s
37.953 s


I started eye.png
47.38 %
1001.191 s
48.078 %
986.515 s
48.071 %
957.426 s
48.002 %
989.542 s
1073.82 s
1119.513 s
1139.691 s
1107.145 s


I started lena.png
47.324 %
9.259 s
48.136 %
8.522 s
48.114 %
8.408 s
48.066 %
7.592 s
12.305 s
13.595 s
13.548 s
13.998 s


I started lena_r.png
47.076 %
61.69 s
47.681 %
53.909 s
47.796 %
53.922 s
47.518 %
53.813 s
65.455 s
69.533 s
70.352 s
69.567 s


I started logo.png
35.652 %
0.457 s
46.502 %
0.615 s
47.221 %
0.696 s
47.06 %
0.603 s
1.655 s
1.627 s
1.46 s
1.377 s


I started Mandrill.png
47.408 %
9.271 s
48.114 %
8.345 s
48.133 %
11.771 s
48.147 %
9.874 s
13.325 s
14.597 s
13.862 s
14.091 s


I started peppers.png
47.439 %
9.507 s
48.183 %
8.389 s
48.189 %
8.545 s
48.207 %
7.79 s
12.583 s
13.649 s
13.847 s
13.917 s


I started scale.png
47.07 %
4.97 s
47.232 %
5.337 s
47.377 %
4.023 s
47.152 %
3.814 s
7.348 s
7.423 s
7.925 s
8.115 s
